In [39]:
import os
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import pandas as pd
import keras as K

In [46]:
def choose_log_file():
    data = list()
    filename = f'./1 - Logs Here/test.log.1'
    file = open(filename, 'r')
    lines = file.read().splitlines()
    file.close()
    for line in lines:
        if not line:
            continue
        substring = '"-"'
        columns2 = [col.strip() for col in line.split('"-"') if col]
        while substring in columns2:
            columns2 = [col.strip() for col in columns2.split('"-"') if col]
        impData = columns2[0]
        impData2 = [col.strip() for col in impData.split(' - - ')]
        try:
            testString = impData2[1].split('"')
        except:
            continue
        date = testString[0].strip(' ')
        request = testString[1]
        requestResponseCode = testString[2].split(' ')

        try:
            requestResponseCodeShortened = requestResponseCode[1]
        except:
            requestResponseCodeShortened = np.nan

        if requestResponseCodeShortened == '200' or requestResponseCodeShortened == '201' or requestResponseCodeShortened == '202':
            columnToAdd = '0'
        else:
            columnToAdd = '1'
        data.append([impData2[0],date,request,requestResponseCodeShortened,columnToAdd])
    print('Log successfully parsed!')

    df = pd.DataFrame(data=data)
    df.columns = ['IP', 'Datetime', 'Request', 'StatusCode', 'Target']
    df = df.dropna()
    return np.array(data), df

def split_dataset(df):
    x = df.values[:,0:4]
    y = df.values[:,4]
    df.values[:,3:5] = df.values[:,3:5].astype(str)

    #encode the x dataset
    # Get unique classes (encuentra los elementos unicos en el array).
    x_dataset_base = x
    y_dataset_base = y
    x_dataset_ips = x_dataset_base[:,0]
    x_dataset_dates = x_dataset_base[:,1]
    uniqueValues = np.unique(x_dataset_ips)
    classes = np.unique(x)
    #for value in x_dataset_ips:

    # Replace classes with integers.
    x = np.searchsorted(classes, x)
    inputColumnQuantityVariables = len(x[0])

    #encode the y dataset
    y = np.array(y.astype(int))

    return x, y, x_dataset_ips, x_dataset_dates

def predict_attacks()    

In [47]:
dataset, df = choose_log_file()
x, y, x_dataset_ips, x_dataset_dates = split_dataset(df)
model = K.models.load_model('./model.h5')


Log successfully parsed!


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 12)                60        
                                                                 
 dense_15 (Dense)            (None, 10)                130       
                                                                 
 dense_16 (Dense)            (None, 8)                 88        
                                                                 
 dense_17 (Dense)            (None, 6)                 54        
                                                                 
 dense_18 (Dense)            (None, 4)                 28        
                                                                 
 dense_19 (Dense)            (None, 4)                 20        
                                                                 
 dense_20 (Dense)            (None, 1)                

In [43]:
predictions = model.predict(x)

possible_attack_hosts = []
done_values = []
bad_requests = 0

for i, element in enumerate(x_dataset_ips):
    previous_element = x_dataset_ips[i - 1] if i > 0 else None
    current_element = element
    if current_element != previous_element and current_element not in done_values:
        done_values.append(current_element)
        times_appeared = np.count_nonzero(x_dataset_ips == current_element)
        lines_in = np.array(np.where(x_dataset_ips == current_element))
        lines_in_added = np.array([a+1 for a in lines_in])
    
        possible_attack_hosts.append((current_element,times_appeared))
        if times_appeared > 100:
#             possible_attack_hosts.append([current_element,times_appeared])
            bad_request_dates = []
            for line_number in lines_in:
                for numb in line_number:
                    if predictions[numb] == 1:
                        #bad_request_lines = array(numpy.where(predictions == predictions[numb]))
                        bad_requests = bad_requests + 1
                    #for date in x_dataset_dates:
                        bad_request_dates.append(x_dataset_dates[numb])
        bad_requests = 0

In [45]:
len(possible_attack_hosts)

1795